In [ ]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=3f8ac13926f7c5fedcbf9b5a8587d14fc1013b925a374894a94604a616afc796
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []
      coverpage=soup.find_all('loc')
      for link in coverpage:
        if link.get_text().find('www.diariodeibiza.es/nacional/')>=0 :
          rows.append(link.get_text()) 
        else :
          if link.get_text().find('www.diariodeibiza.es/internacional/')>=0:
            rows.append(link.get_text()) 
          else :
            if (link.get_text().find('https://www.diariodeibiza.es/pitiusas-y-baleares/')>=0
                or link.get_text().find('https://www.diariodeibiza.es/sucesos/tribunales/')>=0
                or link.get_text().find('https://www.diariodeibiza.es/violencia-de-genero/')>=0
                or link.get_text().find('https://www.diariodeibiza.es/sucesos/delitos-informaticos/')>=0):
              rows.append(link.get_text()) 
    except:
      rows = 'error'
      
    return rows

In [ ]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

url_str=''

try:
  coverpage = requests.get('https://www.diariodeibiza.es/sitemap_index_fdff0.xml').content
  soup = BeautifulSoup(coverpage, 'html.parser')
  
  rows =[]
  coverpage=soup.find_all('loc')
  for link in coverpage:
    rows.append(link.get_text()) 
except:
  rows = 'error'
df = pd.DataFrame(range(0,25),columns = ['page'])
df['page']=rows
print(df)
df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/diariodeibiza/url.csv',index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
                                                                      page
0               https://www.diariodeibiza.es/sitemap_google_news_fdff0.xml
1                     https://www.diariodeibiza.es/sitemap_today_fdff0.xml
2      https://www.diariodeibiza.es/sitemap_contents_2022_08_fdff0_001.xml
3             https://www.diariodeibiza.es/sitemap_community_fdff0_001.xml
4                  https://www.diariodeibiza.es/sitemap_city_fdff0_001.xml
5                   https://www.diariodeibiza.es/sitemap_tag_fdff0_002.xml
6                   https://www.diariodeibiza.es/sitemap_tag_fdff0_001.xml
7               https://www.diariodeibiza.es/sitemap_section_fdff0_001.xml
8             https://www.diariodeibiza.es/sitemap_microsite_fdff0_001.xml
9                https://www.diariodeibiza.es/sitemap_author_fdff0_001.xml
10     https://www.diariodeibiza

                                                                      page  \
0               https://www.diariodeibiza.es/sitemap_google_news_fdff0.xml   
1               https://www.diariodeibiza.es/sitemap_google_news_fdff0.xml   
2               https://www.diariodeibiza.es/sitemap_google_news_fdff0.xml   
3               https://www.diariodeibiza.es/sitemap_google_news_fdff0.xml   
4               https://www.diariodeibiza.es/sitemap_google_news_fdff0.xml   
...                                                                    ...   
15846  https://www.diariodeibiza.es/sitemap_contents_2022_01_fdff0_001.xml   
15847  https://www.diariodeibiza.es/sitemap_contents_2022_01_fdff0_001.xml   
15848  https://www.diariodeibiza.es/sitemap_contents_2022_01_fdff0_001.xml   
15849  https://www.diariodeibiza.es/sitemap_contents_2022_01_fdff0_001.xml   
15850  https://www.diariodeibiza.es/sitemap_contents_2022_01_fdff0_001.xml   

                                                               